## <b><font color='darkblue'>Preface</font></b>
([source](https://learn.deeplearning.ai/courses/langchain/lesson/4/chains)) <b><font size='3ptx'>Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with LCEL. ([more](https://python.langchain.com/v0.1/docs/modules/chains/))</font></b>

<a id='sect0'></a>
### <b><font color='darkgreen'>Outline</font></b>
* <font size='3ptx'><a href='#sect1'>**LLMChain**</a></font> (deprecated in LangChain 0.1.17)
* <font size='3ptx'><a href='#sect2'>**Sequential Chains**</a></font>
    - SimpleSequentialChain
    - SequentialChain
* <font size='3ptx'><a href='#sect3'>**Router Chain**</a></font>

In [4]:
import datetime
import os
import openai
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
import pandas as pd


TEST_DATA = pd.DataFrame({'index': ['row1', 'row2'], 'review': ['review1', 'review2']})

a = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

<a id='sect1'></a>
## <b><font color='darkblue'>LLMChain</font></b>

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [11]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# `LLMChain` was deprecated in LangChain 0.1.17
# chain = LLMChain(llm=llm, prompt=prompt)
chain = prompt | llm

In [14]:
product = "Queen Size Sheet Set"
resp = chain.invoke(product)

In [15]:
resp.content

'"Royal Comfort Linens"'

<a id='sect2'></a>
## <b><font color='darkblue'>SimpleSequentialChain</font></b> ([back](#sect0))
<b><font size='3ptx'>[SimpleSequentialChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SimpleSequentialChain.html) is a structure for combining multiple LLMChains, allowing for sequential processing of different language tasks.</font></b>

<b>It is simpler than [SequentialChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SequentialChain.html) because it directly passes the output of one LLMChain as input to the next, without additional intermediate variables or complex data flow management</b>. This design makes [**SimpleSequentialChain**](https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SimpleSequentialChain.html) ideal for simple sequential tasks, such as first generating a company name and then creating a description for that name, as in this example. It simplifies the implementation of sequential processing steps and is suitable for scenarios that require sequential processing but do not need complex data management.

In a simple scenario with only one input and one output, two LLMChains can be concatenated using [**SimpleSequentialChain**](https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SimpleSequentialChain.html):

![simple chain](images/ch4_img1.PNG)

In [17]:
from langchain.chains import SimpleSequentialChain

In [25]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Assume you are good at math. Please answer below question"
    "Question: {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [24]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Base on result as {result}, is it even?")
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [26]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True)

In [28]:
overall_simple_chain.invoke({'input': 'What is 1 + 2?'})



> Entering new SimpleSequentialChain chain...
The answer is 3.
No, the number 3 is not an even number. It is an odd number.

> Finished chain.


{'input': 'What is 1 + 2?',
 'output': 'No, the number 3 is not an even number. It is an odd number.'}

## <b><font color='darkblue'>SequentialChain</font></b>
<b><font size='3ptx'>[SequentialChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SequentialChain.html) is a structure used to connect multiple LLMChains (<font color='brown'>language model chains</font>) in sequence.</font></b>

In this case, each LLMChain represents a specific processing step (<font color='brown'>such as translation, summarization, etc.</font>), and [**SequentialChain**](https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SequentialChain.html) ensures that these steps are executed in order. <b>This structure allows complex data processing flows to be broken down into smaller, more manageable units and executed sequentially, thereby effectively handling complex language tasks</b>.
![sequential chain](images/ch4_img2.PNG)

In [37]:
from langchain.chains import SequentialChain

llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Count the total money earned by below information: "
    "{Background}"
)
# chain 1: input= Background and output= Salary_sum
chain_one = LLMChain(
    llm=llm, prompt=first_prompt, output_key="Salary_sum")

In [38]:
second_prompt = ChatPromptTemplate.from_template(
    'Double the value of "{Salary_sum}"')

# chain 2: input= English_Review and output= summary
chain_two = LLMChain(
    llm=llm, prompt=second_prompt,  output_key="Final_salary")

In [39]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "Count the people mentioned in below information:\n\n{Background}"
)
# chain 3: input= Background and output= Num_people
chain_three = LLMChain(
    llm=llm, prompt=third_prompt, output_key="Num_people")

In [40]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    'Divide the Final salary as "{Final_salary}" '
    'by number of people as "{Num_people}" and return the result.')

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(
    llm=llm, prompt=fourth_prompt, output_key="Final_result")

In [41]:
# overall_chain: input= Background 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Background"],
    output_variables=["Final_salary", "Num_people", "Final_result"],
    verbose=True
)

In [42]:
background = 'John earned $100; Peter earned $150; Mary earned $200'

In [45]:
overall_chain.invoke({'Background': background})



> Entering new SequentialChain chain...

> Finished chain.


{'Background': 'John earned $100; Peter earned $150; Mary earned $200',
 'Final_salary': 'The revised statement would be: The total money earned is $900. \n\n$200 (John) + $300 (Peter) + $400 (Mary) = $900',
 'Num_people': 'There are three people mentioned: John, Peter, and Mary.',
 'Final_result': 'The average salary per person is $300.'}

<a id='sect3'></a>
## <b><font color='darkblue'>Router Chain</font></b> ([back](#sect0))
<b><font size='3ptx'>The purpose of the Router Chain ([LLMRouterChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.router.llm_router.LLMRouterChain.html)) is to automatically determine which category of template is most suitable for answering a user's question when it is received</font></b>.

For example, if the question is about physics, it will select the physics template to generate the answer. This greatly improves the quality and relevance of the answer.

### <b><font color='darkgreen'>Demo of Router Chain</font></b>
Designed to guide language models to answer questions as an expert in a specific field. This enables the language model to focus more on knowledge in a particular domain and provide answers based on the expertise of that field.

In [46]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [48]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [69]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [77]:
prompt_infos = [
    {
        "name": "Physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template,
    },
    {
        "name": "Math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template,
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template,
    },
    {
        "name": "Computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template,
    },
    {
        "name": "Default",
        "description": "If you don't find a better option, select this option.",
        "prompt_template": "{input}",
    },
]

In [71]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [78]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [79]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising \
it will ultimately lead to a better response from the language model. \
If you don't know which prompt is better, select name as "Default" always.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [82]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [83]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [84]:
chain = MultiPromptChain(
    router_chain=router_chain, 
    destination_chains=destination_chains, 
    default_chain=default_chain,
    verbose=True)

In [81]:
chain.invoke({'input': "What is black body radiation?"})

{'input': 'What is black body radiation?',
 'text': "Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body is characterized by a continuous spectrum of wavelengths and intensities that depend only on the temperature of the body. This phenomenon is described by Planck's law of black body radiation, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and follows a specific distribution known as the Planck distribution. Black body radiation is important in various fields of physics, including thermodynamics, quantum mechanics, and astrophysics."}

In [61]:
chain.invoke({'input': "what is 2 + 2"})



> Entering new MultiPromptChain chain...
Math: {'input': 'what is 2 + 2'}
> Finished chain.


{'input': 'what is 2 + 2', 'text': 'The answer to 2 + 2 is 4.'}

In [86]:
chain.invoke({'input': "What day is today?"})



> Entering new MultiPromptChain chain...
None: {'input': 'What day is today?'}
> Finished chain.


{'input': 'What day is today?', 'text': 'Today is Thursday.'}

## <b><font color='darkblue'>Supplement</font></b>
* [Deeplearning.ai - Langchain Ch2: Model, prompt and parser](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch2_model_prompt_and_parser.ipynb)
* [Deeplearning.ai - Langchain Ch3: Memory](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch3_memory.ipynb)
* [Deeplearning.ai - Langchain Ch4: Chain](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch4_chains.ipynb)
* [Deeplearning.ai - Langchain Ch5: Question and answer](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch5_question_and_answer.ipynb)
* [Deeplearning.ai - Langchain Ch6: Evaluation](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch6_evaluation.ipynb)
* [Deeplearning.ai - Langchain Ch7: Agents](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch7_agents.ipynb)
* [Medium - LangChain — Sequential LLM Calls](https://medium.com/@princekrampah/langchain-sequential-6912164ca568)
* [[LangChain for LLM Application Development] 課程筆記- Chains](https://hackmd.io/@YungHuiHsu/SJJvZ-ya2?utm_source=preview-mode&utm_medium=rec)